In [76]:
import kagglehub
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

path_pop = kagglehub.dataset_download("iamsouravbanerjee/world-population-dataset")
df_pop = pd.read_csv(f"{path_pop}/world_population.csv")

# 2. Annual CO2 Emissions by Country
path_co2 = kagglehub.dataset_download("ulrikthygepedersen/co2-emissions-by-country")
df_co2 = pd.read_csv(f"{path_co2}/co2_emissions_kt_by_country.csv")

# 3. Monthly Climate Data by Station
path_station = kagglehub.dataset_download("christopherlemke/monthly-climat-reports-from-stations-worldwide")
df_reports = pd.read_csv(f"{path_station}/dwd-cdc_CLIMAT_reports_stations_ww.csv")
df_stations = pd.read_csv(f"{path_station}/dwd-cdc_station_data_ww.csv")

path_aq1 = kagglehub.dataset_download("kanchana1990/world-air-quality-data-2024-updated")
df_aq1 = pd.read_csv(f"{path_aq1}/world_air_quality.csv", on_bad_lines='skip', sep=';')

path_aq2 = kagglehub.dataset_download("dnkumars/air-quality-index")
df_aq2 = pd.read_csv(f"{path_aq2}/cleaned_interpolated_dataset.csv")

Using Colab cache for faster access to the 'world-population-dataset' dataset.
Using Colab cache for faster access to the 'co2-emissions-by-country' dataset.
Using Colab cache for faster access to the 'monthly-climat-reports-from-stations-worldwide' dataset.
Using Colab cache for faster access to the 'world-air-quality-data-2024-updated' dataset.
Using Colab cache for faster access to the 'air-quality-index' dataset.


## Co2 DF

In [77]:
co2_countries = df_co2['country_code'].unique()
pop_countries = df_pop['CCA3'].unique()

In [78]:
country_map = df_pop.loc[:, ['CCA3', 'Country/Territory']].set_index('CCA3')['Country/Territory']
country_map.to_csv('country_map.csv')
country_map

,Country/Territory
CCA3,
AFG,Afghanistan
ALB,Albania
DZA,Algeria
ASM,American Samoa
AND,Andorra
...,...
WLF,Wallis and Futuna
ESH,Western Sahara
YEM,Yemen


In [79]:
missing_countries = set(co2_countries) - set(pop_countries)
df_co2[df_co2['country_code'].isin(missing_countries)]['country_name'].unique()


array(['Africa Eastern and Southern', 'Africa Western and Central',
       'Arab World', 'Central Europe and the Baltics',
       'Caribbean small states',
       'East Asia & Pacific (excluding high income)',
       'Early-demographic dividend', 'East Asia & Pacific',
       'Europe & Central Asia (excluding high income)',
       'Europe & Central Asia', 'Euro area', 'European Union',
       'Fragile and conflict affected situations', 'High income',
       'Heavily indebted poor countries (HIPC)', 'IBRD only',
       'IDA & IBRD total', 'IDA total', 'IDA blend', 'IDA only',
       'Latin America & Caribbean (excluding high income)',
       'Latin America & Caribbean',
       'Least developed countries: UN classification', 'Low income',
       'Lower middle income', 'Low & middle income',
       'Late-demographic dividend', 'Middle East & North Africa',
       'Middle income',
       'Middle East & North Africa (excluding high income)',
       'North America', 'OECD members', 'Other sm

In [80]:

df_co2 = df_co2[~df_co2['country_code'].isin(missing_countries)]


In [81]:
# Step 1: Prepare df_pop in long format
df_pop_long = df_pop.melt(
    id_vars=['CCA3'],
    value_vars=['1970 Population', '1980 Population', '1990 Population', '2000 Population',
                '2010 Population', '2015 Population', '2020 Population', '2022 Population'],
    var_name='year',
    value_name='population'
)

# Clean and convert year to integer
df_pop_long['year'] = df_pop_long['year'].str.extract(r'(\d+)').astype(int)

all_years = np.arange(df_pop_long['year'].min(), df_pop_long['year'].max() + 1)

df_list = []
for country, group in df_pop_long.groupby('CCA3'):
    group = group.set_index('year').reindex(all_years)
    group['CCA3'] = country
    group['population'] = group['population'].interpolate(method='linear')
    df_list.append(group.reset_index())

df_pop_interp = pd.concat(df_list, ignore_index=True)

df_pop_interp

,year,CCA3,population
0,1970,ABW,59106.0
1,1971,ABW,59422.1
2,1972,ABW,59738.2
3,1973,ABW,60054.3
4,1974,ABW,60370.4
...,...,...,...
12397,2018,ZWE,15063774.4
12398,2019,ZWE,15366720.2
12399,2020,ZWE,15669666.0
12400,2021,ZWE,15995101.5


In [82]:
# Step 3: Merge safely
df_co2 = df_co2.merge(
    df_pop_interp,
    how='inner',
    left_on=['country_code', 'year'],
    right_on=['CCA3', 'year']
).drop(columns=['CCA3'])


In [83]:
df_co2['co2_per_capita'] = df_co2['value'] / df_co2['population']
df_co2 = df_co2.drop(columns=["country_name", "population"])
df_co2.rename(columns={'value':'co2'})

df_co2.to_csv('co2.csv')
print(df_co2.isna().sum())
df_co2.head()

country_code      0
year              0
value             0
co2_per_capita    0
dtype: int64


,country_code,year,value,co2_per_capita
0,ABW,1970,16655.514,0.281791
1,ABW,1971,14495.651,0.243944
2,ABW,1972,14055.611,0.235287
3,ABW,1973,15592.084,0.259633
4,ABW,1974,14132.618,0.234098


## Climate DF

In [84]:
# --- Step 0: Rename station columns ---
df_stations = df_stations.rename(columns={
    '0': 'id',
    '1': 'name',
    '2': 'latitude',
    '3': 'longitude',
    '4': 'height',
    '5': 'country'
})

# --- Step 1: Keep only rows with non-missing, numeric IDs ---
df_reports = df_reports[pd.to_numeric(df_reports["IIiii"], errors='coerce').notna()].copy()
df_stations = df_stations[pd.to_numeric(df_stations["id"], errors='coerce').notna()].copy()

# --- Step 2: Convert to int → str ---
df_reports["IIiii"] = df_reports["IIiii"].astype(float).astype(int).astype(str)
df_stations["id"] = df_stations["id"].astype(float).astype(int).astype(str)

# --- Step 3: Check overlap ---
reports_ids = set(df_reports["IIiii"])
stations_ids = set(df_stations["id"])
overlap_ids = reports_ids & stations_ids
print(f"Overlapping IDs: {len(overlap_ids)}")

# --- Step 4: Merge using inner join ---
df_climate = pd.merge(
    df_reports,
    df_stations,
    left_on="IIiii",
    right_on="id",
    how="inner"
).drop(columns=['id'])

print(df_climate.head())



Overlapping IDs: 3458
     year  month IIiii   G1       Po  G1.1        P  G1.2   sn     T  ...  \
0  2013.0    4.0  1001  1.0  10095.0   2.0  10107.0   3.0  1.0  42.0  ...   
1  2013.0    4.0  1007  1.0  10084.0   2.0  10094.0   3.0  1.0  92.0  ...   
2  2013.0    4.0  1008  1.0  10056.0   2.0  10091.0   3.0  1.0  93.0  ...   
3  2013.0    4.0  1025  1.0  10043.0   2.0  10054.0   3.0  0.0  10.0  ...   
4  2013.0    4.0  1026  1.0   9912.0   2.0  10054.0   3.0  0.0  12.0  ...   

   Dgr  G4.7  iy  Gx  Gn                      name  \
0  NaN   NaN NaN NaN NaN                 Jan Mayen   
1  NaN   NaN NaN NaN NaN                Ny-Alesund   
2  NaN   NaN NaN NaN NaN                  Svalbard   
3  NaN   NaN NaN NaN NaN            Tromso/Langnes   
4  NaN   NaN NaN NaN NaN                    Tromso   

                                        latitude   longitude   height  \
0                                          70.94      -08.67        9   
1                                          7

In [85]:
import pandas as pd

# Clean country names
df_climate['country'] = df_climate['country'].str.replace('\r\n', '').str.strip()


df_climate = df_climate.loc[:, ['country', 'year', 'month','sn', 'T', 'R1', 'sn.1', 'Tx', 'sn.2', 'Tn']]
df_climate['temp_mean'] = (1 - 2 * df_climate['sn']) * df_climate['T'] / 10
df_climate['temp_max']  = (1 - 2 * df_climate['sn.1']) * df_climate['Tx'] / 10
df_climate['temp_min']  = (1 - 2 * df_climate['sn.2']) * df_climate['Tn']  / 10# assuming sn.2 also applies to Tn


# Step 1: Compute monthly mean for each year-country-month
df_climate = (
    df_climate
    .groupby(['year', 'country', 'month'], as_index=False)
    .mean()  # averages sn, T, R1, temp
)

# Compute signed temperatures

# Optional: drop original columns if no longer needed
df_climate = df_climate.drop(columns=['T', 'Tx', 'Tn', 'sn', 'sn.1', 'sn.2'])

df_climate['year'] = df_climate['year'].astype(int)
df_climate['month'] = df_climate['month'].astype(int)
# Check result
df_climate.head()


,year,country,month,R1,temp_mean,temp_max,temp_min
0,2003,Algeria,3,16.200000,15.023529,20.605882,9.917647
1,2003,Algeria,4,1025.634146,18.173171,24.068293,12.251220
2,2003,Algeria,5,545.631579,21.707018,27.944643,15.366667
3,2003,Algeria,6,1760.052632,27.694828,34.146552,21.162069
4,2003,Algeria,7,447.311111,30.135556,36.644444,23.735556


In [86]:
print(set(df_climate['country'].unique()) - set(country_map.unique()))
print(set(country_map.unique()) - set(df_climate['country'].unique()))

{'', 'Australien, SW-Pazifik', 'Cook-Island', 'United States of America', 'Caroline-Islands', 'Tunesia', 'St. Maarten', 'Croatia/Hrvatska', 'Republic of China, Taiwan', 'Slowenia', 'Indonesien', 'Palau-Islands', "Cote d'Ivoire", 'Russian Federation', "Korea, Dem. People's Rep.", 'Iran (Islamic Rep. of)', 'Mauretania', 'Marshall-Islands', 'Macedonia', 'United Arab. Emirates', 'Republic of Korea', 'United Kingdom of Great Britain and N.-Ireland', "People's Dem. Rep. Laos", 'Bosnia and Herzegowina', 'Ascencion Island', 'Syrian Arab Rep.', 'Dem. Republic of the Congo', 'Slovakia (Slovak. Rep.)', 'Western-Sahara', 'Wake-Insel', 'Moldova, Rep. Of', 'Wallis-Islands'}
{'Mauritania', 'Estonia', 'Turks and Caicos Islands', 'Northern Mariana Islands', 'Sint Maarten', 'North Korea', 'Palau', 'Laos', 'DR Congo', 'Hong Kong', 'Djibouti', 'Aruba', 'Monaco', 'Lithuania', 'Falkland Islands', 'Mayotte', 'United Arab Emirates', 'Saint Vincent and the Grenadines', 'North Macedonia', 'Tokelau', 'Burundi', 

In [87]:
country_corrections = {
    'Australien, SW-Pazifik': 'Australia',
    'Cook-Island': 'Cook Islands',
    'United States of America': 'United States',
    'Caroline-Islands': 'Micronesia',
    'Tunesia': 'Tunisia',
    'St. Maarten': 'Sint Maarten',
    'Croatia/Hrvatska': 'Croatia',
    'Republic of China, Taiwan': 'Taiwan',
    'Slowenia': 'Slovenia',
    'Indonesien': 'Indonesia',
    'Palau-Islands': 'Palau',
    "Cote d'Ivoire": 'Ivory Coast',
    'Russian Federation': 'Russia',
    "Korea, Dem. People's Rep.": 'North Korea',
    'Iran (Islamic Rep. of)': 'Iran',
    'Mauretania': 'Mauritania',
    'Marshall-Islands': 'Marshall Islands',
    'Macedonia': 'North Macedonia',
    'United Arab. Emirates': 'United Arab Emirates',
    'Republic of Korea': 'South Korea',
    'United Kingdom of Great Britain and N.-Ireland': 'United Kingdom',
    "People's Dem. Rep. Laos": 'Laos',
    'Bosnia and Herzegowina': 'Bosnia and Herzegovina',
    'Ascencion Island': 'Ascension Island',
    'Syrian Arab Rep.': 'Syria',
    'Dem. Republic of the Congo': 'DR Congo',
    'Slovakia (Slovak. Rep.)': 'Slovakia',
    'Western-Sahara': 'Western Sahara',
    'Wake-Insel': 'Wake Island',
    'Moldova, Rep. Of': 'Moldova',
    'Wallis-Islands': 'Wallis and Futuna',
    '': None  # optional: empty string to None
}
df_climate['country'] = df_climate['country'].replace(country_corrections)

# country_map: index = code, values = country_name
# Create a reversed mapping: country_name -> code
reversed_map = pd.Series(country_map.index, index=country_map.values)

# Now map the country names to codes
df_climate['country_code'] = df_climate['country'].map(reversed_map)
print(f"NA codes: {df_climate.isna().sum()}")

df_climate = df_climate.drop(columns = 'country')
df_climate = df_climate.dropna(subset=["country_code", "temp_mean"])

df_climate.to_csv('climate.csv')
df_climate.head()

NA codes: year               0
country          159
month              0
R1               210
temp_mean        328
temp_max        1141
temp_min        1151
country_code     392
dtype: int64


,year,month,R1,temp_mean,temp_max,temp_min,country_code
0,2003,3,16.200000,15.023529,20.605882,9.917647,DZA
1,2003,4,1025.634146,18.173171,24.068293,12.251220,DZA
2,2003,5,545.631579,21.707018,27.944643,15.366667,DZA
3,2003,6,1760.052632,27.694828,34.146552,21.162069,DZA
4,2003,7,447.311111,30.135556,36.644444,23.735556,DZA


## AQI DFs

### AQ1: Various Pollutants

In [88]:
df_aq1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54255 entries, 0 to 54254
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Country Code   54255 non-null  object 
 1   City           30209 non-null  object 
 2   Location       54253 non-null  object 
 3   Coordinates    54185 non-null  object 
 4   Pollutant      54255 non-null  object 
 5   Source Name    54255 non-null  object 
 6   Unit           54255 non-null  object 
 7   Value          54255 non-null  float64
 8   Last Updated   54255 non-null  object 
 9   Country Label  54140 non-null  object 
dtypes: float64(1), object(9)
memory usage: 4.1+ MB


In [89]:
units = df_aq1.groupby('Pollutant')['Unit'].agg(lambda x: x.value_counts().idxmax())

df_aq1 = df_aq1[df_aq1['Unit'] == df_aq1['Pollutant'].map(units)]

means_by_country = df_aq1.groupby(['Country Label', 'Pollutant', 'Unit'])['Value'].mean()

means_by_country.to_csv('pollution.csv')


### AQ2: AQI by country by Month

In [90]:
cols = [c for c in df_aq2.columns if c not in ['rank', 'city', 'city_name', 'Country']]
df_aq = df_aq2.groupby('Country')[cols].mean()

df_aq.to_csv('air_quality.csv')
df_aq.sort_values('avg', ascending=False).head()

,avg,jan,feb,mar,apr,may,jun,jul,aug,sep,oct,nov,dec
Country,,,,,,,,,,,,,
Bangladesh,161.000000,303.000000,195.000000,184.000000,116.000000,125.000000,100.000000,51.000000,107.000000,83.000000,158.000000,223.000000,284.000000
Chad,147.000000,221.000000,326.000000,189.000000,285.000000,106.000000,87.500000,58.500000,38.000000,35.000000,207.000000,228.000000,235.000000
Kuwait,95.333333,90.000000,96.000000,72.666667,78.333333,111.333333,117.666667,92.333333,110.000000,108.666667,97.666667,81.333333,86.666667
India,83.283372,132.065574,109.847775,85.391101,77.925059,67.594848,62.161593,42.587822,55.494145,48.189696,81.800937,114.119438,121.674473
Pakistan,79.166667,124.500000,82.333333,61.833333,42.666667,36.666667,41.833333,33.000000,41.222222,45.555556,77.666667,177.500000,191.666667
